In [2]:
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torchvision
import glob

In [1]:
imgs = []

In [4]:
# Show all images
def show_imgs():
    for img in imgs:
        cv.imshow(img, cv.imread(img))
        cv.waitKey(0)
        cv.destroyWindow(img)

# Grayscale
def show_grayscale():
    for img in imgs:
        gray = cv.cvtColor(cv.imread(img), cv.COLOR_BGR2GRAY)
        cv.imshow(img, gray)
        cv.waitKey(0)
        cv.destroyWindow(img)

def show_canny():
    for img in imgs:
        canny = cv.Canny(cv.imread(img), 100, 200)
        cv.imshow(img, canny)
        cv.waitKey(0)
        cv.destroyWindow(img)

In [2]:
# THESE DO NOT WORK AT THE MOMENT
show_imgs()
show_grayscale()
show_canny()

NameError: name 'show_imgs' is not defined

In [7]:
class NNet(nn.Module):
    def __init__(self, learning_rate, epochs, batch_size, classes=5):
        super(NNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32*2*2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, 16)
        self.pool = nn.MaxPool2d(2, 2)
    
    # activation
    def forward(self, x):
        # RELU first convolution, pooling it
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x) # 2x2 kernel to "scan" image
        
        # second convolution

        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        
        # hidden linear layer
        x = x.reshape(-1, 32*2*2)
        x = nn.functional.relu(self.fc1(x))

        x = nn.functional.relu(self.fc2(x))

        return nn.functional.softmax(self.out(x), dim=1)
    
    

In [ ]:
cnn = NNet(imgs)

cross_entropy_loss = nn.functional.cross_entropy()